In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import pandas as pd
pd.DataFrame(list(range(10)))

,0
0,0
1,1
2,2
3,3
4,4
5,5
6,6
7,7
8,8
9,9


In [2]:
from yieldcurves.models import HullWhite

# HullWhite

In [ ]:
hw = HullWhite(mean_reversion=0.1, volatility=0.01, terminal_date=10.).curve(0.02)
f = HullWhite(mean_reversion=0.01, volatility=0.02, domestic=hw.model).curve(0.03)
fx = f.model.fx(1.1)

In [ ]:
hw.evolve()
print(f"{hw=}")
print(dict(hw.items()))
print(f"{hw(3)=}")

In [ ]:
f.evolve()
print(f"{f=}")
print(dict(f.items()))
print(f"{f(3)=}")

In [ ]:
fx.evolve()
print(f"{fx=}")
print(dict(fx.items()))
print(f"{float(fx)=}")

In [44]:
hw.model.increments(8, step_size=.9)

[0.9, 1.8, 2.7, 3.6, 4.5, 5.4, 6.3, 7.2]

In [40]:
sample = hw.sample(3)
sample

[HullWhite.Curve(0.02, model=HullWhite(0.1, 0.01, 10.0)),
 HullWhite.Curve(0.02, model=HullWhite(0.1, 0.01, 10.0)),
 HullWhite.Curve(0.02, model=HullWhite(0.1, 0.01, 10.0))]

In [41]:
[list(c.values()) for c in sample]

[[0.0020565393228650472, -0.019935372045203004],
 [0.0020565393228650472, -0.019935372045203004, -0.025770784502536195],
 [0.0020565393228650472,
  -0.019935372045203004,
  -0.025770784502536195,
  -0.04167574374639585]]

In [43]:
[c(1.5) for c in sample]

[0.0004964805508022606, -0.0006430466369076481, -0.0023343784398829094]

In [ ]:
hw.clear()
dict(hw.items())

In [ ]:
hw.evolve(0.25)
dict(hw.items())

In [45]:
hw.simulate(3, 2)

[[HullWhite.Curve(0.02, model=HullWhite(0.1, 0.01, 10.0)),
  HullWhite.Curve(0.02, model=HullWhite(0.1, 0.01, 10.0))],
 [HullWhite.Curve(0.02, model=HullWhite(0.1, 0.01, 10.0)),
  HullWhite.Curve(0.02, model=HullWhite(0.1, 0.01, 10.0))],
 [HullWhite.Curve(0.02, model=HullWhite(0.1, 0.01, 10.0)),
  HullWhite.Curve(0.02, model=HullWhite(0.1, 0.01, 10.0))]]

In [ ]:
[dict(_[1].items()) for _ in hw.simulate(3, 2, step_size=.25)]

# Global

In [45]:
rate_corr = [[1.0, 0.6], [0.6, 1.0]]
fx_corr = [[1.0]]
rate_fx_corr = [[0.22], [0.33]]

In [46]:
# 1. correlation
corr = HullWhite.Global.foreign_correlation(rate_corr, fx_corr, rate_fx_corr)
corr

[[1.0, 0.6, 0.22], [0.6, 1.0, 0.33], [0.22, 0.33, 1.0]]

In [42]:
rate_curve = 0.02, 0.03
fx_spot = 1.2, 2.3

mean_reversion = 0.12, 0.21
volatility = 0.06, 0.07
fx_volatility = 0.1, 0.2

In [43]:
# 2. factors
domestic = HullWhite(0.1, 0.04).curve(0.01)
factors = HullWhite.Global.foreign_factors(rate_curve, fx=fx_spot, mean_reversion=mean_reversion, volatility=volatility, domestic=domestic, fx_volatility=())
factors

[HullWhite.Curve(0.02, model=HullWhite(0.12, 0.06, domestic=HullWhite(0.1, 0.04))),
 HullWhite.Fx(1.2, model=HullWhite(0.12, 0.06, domestic=HullWhite(0.1, 0.04))),
 HullWhite.Curve(0.03, model=HullWhite(0.21, 0.07, domestic=HullWhite(0.1, 0.04))),
 HullWhite.Fx(2.3, model=HullWhite(0.21, 0.07, domestic=HullWhite(0.1, 0.04)))]

In [50]:
# 3. init
g = HullWhite.Global([domestic] + factors, correlation=corr)
g

HullWhite.Global((HullWhite.Curve(0.01, model=HullWhite(0.1, 0.04)), HullWhite.Fx(1.2, model=HullWhite(0.12, 0.06, domestic=HullWhite(0.1, 0.04))), HullWhite.Curve(0.02, model=HullWhite(0.12, 0.06, domestic=HullWhite(0.1, 0.04))), HullWhite.Fx(2.3, model=HullWhite(0.21, 0.07, domestic=HullWhite(0.1, 0.04))), HullWhite.Curve(0.03, model=HullWhite(0.21, 0.07, domestic=HullWhite(0.1, 0.04)))), [[1.0, 0.11, 0.6, 0.12, 0.8], [0.11, 1.0, 0.21, 0.3, 0.31], [0.6, 0.21, 1.0, 0.22, 0.7], [0.12, 0.3, 0.22, 1.0, 0.32], [0.8, 0.31, 0.7, 0.32, 1.0]])

In [51]:
# or
g = HullWhite.Global([domestic] + factors)
g.correlation = corr
g

HullWhite.Global((HullWhite.Curve(0.01, model=HullWhite(0.1, 0.04)), HullWhite.Fx(1.2, model=HullWhite(0.12, 0.06, domestic=HullWhite(0.1, 0.04))), HullWhite.Curve(0.02, model=HullWhite(0.12, 0.06, domestic=HullWhite(0.1, 0.04))), HullWhite.Fx(2.3, model=HullWhite(0.21, 0.07, domestic=HullWhite(0.1, 0.04))), HullWhite.Curve(0.03, model=HullWhite(0.21, 0.07, domestic=HullWhite(0.1, 0.04)))), [[1.0, 0.11, 0.6, 0.12, 0.8], [0.11, 1.0, 0.21, 0.3, 0.31], [0.6, 0.21, 1.0, 0.22, 0.7], [0.12, 0.3, 0.22, 1.0, 0.32], [0.8, 0.31, 0.7, 0.32, 1.0]])

In [52]:
# fails
HullWhite.Global(factors)

ValueError: domestic model must not have domestic model HullWhite.Fx(1.2, model=HullWhite(0.12, 0.06, domestic=HullWhite(0.1, 0.04)))

In [53]:
# fails
HullWhite.Global([domestic] + 2* factors, corr)

ValueError: correlation of wrong size 5 expected size 8

In [55]:
g.evolve(1.)
for f in g.factors:
    display(dict(f.items()))

{1.0: -0.048202877025125865, 2.0: -0.04077069919990645}

{1.0: 7.557708070632385e-05, 2.0: 0.029761169186679624}

{1.0: -0.051564265537333384, 2.0: -0.10434215161866081}

{1.0: 0.029738003628753392, 2.0: 0.0590055590704071}

{1.0: -0.10910373818017309, 2.0: -0.14591426321997517}

In [56]:
g.clear()
for f in g.factors:
    display(dict(f.items()))

{}

{}

{}

{}

{}

In [57]:
domestic = HullWhite(0.1, 0.04).curve(0.001)
h = HullWhite.Global.from_parameters(rate_curve, fx=fx_spot, mean_reversion=mean_reversion, volatility=volatility, terminal_date=10., domestic=domestic, fx_volatility=fx_volatility, correlation=corr)

In [63]:
h.evolve()
for f in h.factors:
    display(dict(f.items()))

{0.25: -0.002897153944267994,
 0.5: 0.0027067366924283364,
 0.75: -0.029253603145671877,
 1.0: -0.056753885995787214,
 1.25: -0.045600230236564544,
 1.5: -0.03583115290336188}

{0.25: 0.00822850321955753,
 0.5: 0.019763115313283404,
 0.75: -0.0062152988595186275,
 1.0: -0.020970929832807354,
 1.25: -0.02311150472298121,
 1.5: -0.02797971738831065}

{0.25: 0.013510173842585554,
 0.5: 0.032708854110190706,
 0.75: -0.044109547738164466,
 1.0: -0.024442762173454733,
 1.25: -0.027555176085646352,
 1.5: -0.041878395809740114}

{0.25: 0.03163189032768787,
 0.5: 0.06598034392869835,
 0.75: 0.026885713463399803,
 1.0: 0.05986584234013502,
 1.25: -0.008360549363883895,
 1.5: -0.04625367651352249}

{0.25: 0.012333980153919986,
 0.5: 0.0399540663964371,
 0.75: -0.015031785436183895,
 1.0: -0.0700746562453503,
 1.25: -0.058003353777825895,
 1.5: -0.07363270150529956}

# Random

In [30]:
from sympy import Matrix, sqrt
from sympy.abc import a, b, c
C = Matrix([[1, a, b], [a, 1, c], [b, c, 1]])
C.cholesky(hermitian=False)

Matrix([
[1,                         0,                                          0],
[a,            sqrt(1 - a**2),                                          0],
[b, (-a*b + c)/sqrt(1 - a**2), sqrt(-b**2 + 1 - (-a*b + c)**2/(1 - a**2))]])

In [25]:
from scipy.linalg import cholesky
from math import sqrt

In [32]:
d = sqrt(1 - a**2)
e = (c - a * b) / d if d else 0
f = sqrt(1 - b**2 - e**2)
c1 = 1, 0, 0
c2 = a, d, 0
c3 = b, e, f
Matrix([c1, c2, c3])
q = 1,1,1
x = q[0]
y = a * q[0] + d * q[1]
z = b * q[0] + e * q[1] + f * q[2]

q[0] = x
if d:
    q[1] = (y - a * y[0]) / d
else:
    q[1] = q[0]
if f:
    q[2] = (z - b * q[0] - e * q[1]) / f
else:
    q[2] = 0
    q[1] = (y - a * y[0]) / d
if b == 1:
    q[2] = q[0]
elif f == 0:  # aka c = sqrt(1-b**2) * sqrt(1-a**2) + ab
    q[1] = (z - b * y[0]) / e
else:


Matrix([
[1,                         0,                                          0],
[a,            sqrt(1 - a**2),                                          0],
[b, (-a*b + c)/sqrt(1 - a**2), sqrt(-b**2 + 1 - (-a*b + c)**2/(1 - a**2))]])

In [0]:
b**2 + (c - ab)**2 / (1 - a**2) = 1
(c-ab)**2 = (1-b**2) * (1-a**2) = 1 - b**2 - a**2 + ab**2
c - ab = sqrt(1-b**2) * sqrt(1-a**2)
c = sqrt(1-b**2) * sqrt(1-a**2) + ab

In [24]:


a, b, c = 0.4, 0.3, 0.1
display(cholesky([[1, a, b], [a, 1, c], [b, c, 1]]))
c1 = 1, 0, 0
c2 = a, sqrt(1 - a**2), 0
c3 = b, 0 ,0
display(Matrix([c1, c2, c3]).T)

array([[ 1.        ,  0.4       ,  0.3       ],
       [ 0.        ,  0.91651514, -0.02182179],
       [ 0.        ,  0.        ,  0.95368958]])

Matrix([
[1,               0.4, 0.3],
[0, 0.916515138991168,   0],
[0,                 0,   0]])

In [0]:
a, b = HullWhite.random.gauss(0., 1.), HullWhite.random.gauss(0., 1.)

In [35]:
dim = 5
s = [0] + [i * 2 + 1 for i in range(dim - 1)] + [j * 2 + 2 for j in range(dim - 1)]
s, len(s), dim + dim - 1

([0, 1, 3, 5, 7, 2, 4, 6, 8], 9, 9)

# Golbal 2

In [54]:
from yieldcurves.models import HullWhite

rate_corr = [[1.0, 0.6], [0.6, 1.0]]
fx_corr = [[1.0]]
rate_fx_corr = [[0.22], [0.33]]
corr = HullWhite.Global.foreign_correlation(rate_corr, fx_corr, rate_fx_corr)

domestic = HullWhite(0.1, 0.05).curve(0.02)
foreign =  HullWhite(0.2, 0.02, domestic=domestic, fx_volatility=0.2).curve(0.05)
fx = foreign.model.fx(2.2)

g = HullWhite.Global([domestic, foreign, fx], correlation=corr)

In [55]:
foreign.model.domestic.model == domestic.model

True